# Generate Serialized Space

Assembles the final `serialized_space.json` from component files generated by `nb_export_genie_assets.ipynb`.

## Input Files
Reads from `exports/genie/components/`:
- `sample_questions.json` - Quick-start questions
- `data_sources.json` - Tables with column configs
- `text_instructions.json` - Genie instructions
- `example_queries.json` - Example SQL queries

## Output
- `serialized_space.json` - Complete Genie space configuration

## Usage
This notebook runs as Task 2 in `job_cdm_export_genie_assets`, after the component files are generated.


In [ ]:
# Cell 1: Get Configuration from Setup Task
import json
from datetime import datetime

# Get globals from setup task
try:
    setup_result = dbutils.jobs.taskValues.get(taskKey="setup", key="globals")
    globals_dict = json.loads(setup_result) if isinstance(setup_result, str) else setup_result
    print(f"✓ Loaded globals from setup task")
except Exception as e:
    print(f"⚠ Could not load from setup task: {e}")
    globals_dict = {
        "catalog": "aira_test",
        "bronze_schema": "bronze_md",
        "silver_schema": "silver_md",
        "gold_schema": "gold_md"
    }

# Extract configuration
catalog = globals_dict.get("catalog", "aira_test")
bronze_schema = globals_dict.get("bronze_schema", "bronze_md")

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")


In [ ]:
# Cell 2: Get Job Parameters and Build Paths

# Widget parameters (passed from job)
dbutils.widgets.text("source_root", "test", "Source Root")

source_root = dbutils.widgets.get("source_root")

# Get pipeline config for source_volume
try:
    pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
    source_volume = pipeline_config.get('source_volume', 'clinical_data_standards')
    print(f"✓ Loaded pipeline config from setup task")
except Exception as e:
    print(f"⚠ Could not load pipeline config: {e}")
    source_volume = 'clinical_data_standards'

# Build paths
export_base = f"/Volumes/{catalog}/{bronze_schema}/{source_volume}/{source_root}/exports/genie"
components_dir = f"{export_base}/components"

print(f"="*60)
print(f"GENERATE SERIALIZED SPACE")
print(f"="*60)
print(f"Source Volume: {source_volume}")
print(f"Source Root: {source_root}")
print(f"Export Base: {export_base}")
print(f"Components Dir: {components_dir}")
print(f"="*60)


In [ ]:
# Cell 3: Read Component Files

def read_json_file(filepath):
    """Read and parse a JSON file."""
    with open(filepath, 'r') as f:
        return json.load(f)

# Read all component files
print("📖 Reading component files...")

sample_questions = read_json_file(f"{components_dir}/sample_questions.json")
print(f"  ✓ sample_questions.json: {len(sample_questions)} questions")

data_sources = read_json_file(f"{components_dir}/data_sources.json")
print(f"  ✓ data_sources.json: {len(data_sources['tables'])} tables")

text_instructions = read_json_file(f"{components_dir}/text_instructions.json")
print(f"  ✓ text_instructions.json: {len(text_instructions)} instruction(s)")

example_queries = read_json_file(f"{components_dir}/example_queries.json")
print(f"  ✓ example_queries.json: {len(example_queries)} queries")

print(f"\n✅ All component files loaded successfully")


In [ ]:
# Cell 4: Assemble serialized_space.json

print("🔧 Assembling serialized_space.json...")

# Build the complete serialized space structure
# Format matches Databricks Genie Space API requirements
serialized_space = {
    "version": 1,
    "config": {
        "sample_questions": sample_questions
    },
    "data_sources": data_sources,
    "instructions": {
        "text_instructions": text_instructions,
        "example_question_sqls": example_queries
    }
}

# Write to file
serialized_space_path = f"{export_base}/serialized_space.json"
with open(serialized_space_path, 'w') as f:
    json.dump(serialized_space, f, indent=2)

print(f"✓ Generated: {serialized_space_path}")

# Calculate size
import os
file_size = os.path.getsize(serialized_space_path)
print(f"  Size: {file_size:,} bytes")

# Summary
print(f"\n📋 Serialized Space Summary:")
print(f"  Version: {serialized_space['version']}")
print(f"  Sample Questions: {len(serialized_space['config']['sample_questions'])}")
print(f"  Tables: {len(serialized_space['data_sources']['tables'])}")
print(f"  Text Instructions: {len(serialized_space['instructions']['text_instructions'])}")
print(f"  Example SQLs: {len(serialized_space['instructions']['example_question_sqls'])}")


In [ ]:
# Cell 5: Set Task Values and Complete

result = {
    "success": True,
    "serialized_space_path": serialized_space_path,
    "file_size_bytes": file_size,
    "tables_count": len(serialized_space['data_sources']['tables']),
    "sample_questions_count": len(serialized_space['config']['sample_questions']),
    "example_queries_count": len(serialized_space['instructions']['example_question_sqls']),
    "generated_timestamp": datetime.now().isoformat()
}

dbutils.jobs.taskValues.set(key="serialized_space_result", value=json.dumps(result))

print("\n" + "="*60)
print("SERIALIZED SPACE GENERATION COMPLETE")
print("="*60)
print(f"Output: {serialized_space_path}")
print(f"Size: {file_size:,} bytes")
print("="*60)

dbutils.notebook.exit(json.dumps(result, indent=2))
